In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/kpm-data/key_points_test.csv
/kaggle/input/kpm-data/arguments_dev.csv
/kaggle/input/kpm-data/key_points_train.csv
/kaggle/input/kpm-data/readme.txt
/kaggle/input/kpm-data/labels_test.csv
/kaggle/input/kpm-data/arguments_test.csv
/kaggle/input/kpm-data/labels_train.csv
/kaggle/input/kpm-data/labels_dev.csv
/kaggle/input/kpm-data/key_points_dev.csv
/kaggle/input/kpm-data/arguments_train.csv


In [2]:
pip install transformers accelerate sentencepiece

Note: you may need to restart the kernel to use updated packages.


In [3]:
from transformers import T5ForConditionalGeneration, T5Tokenizer
import pandas as pd
from transformers import GenerationConfig
import torch
device=torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [4]:
model_name = 'jbochi/madlad400-3b-mt'
model = T5ForConditionalGeneration.from_pretrained(model_name, device_map="auto")
tokenizer = T5Tokenizer.from_pretrained(model_name)
model.to(device)

config.json:   0%|          | 0.00/749 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/11.8G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/830 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.43M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/4.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/16.6M [00:00<?, ?B/s]

T5ForConditionalGeneration(
  (shared): Embedding(256000, 1024)
  (encoder): T5Stack(
    (embed_tokens): Embedding(256000, 1024)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=1024, out_features=2048, bias=False)
              (k): Linear(in_features=1024, out_features=2048, bias=False)
              (v): Linear(in_features=1024, out_features=2048, bias=False)
              (o): Linear(in_features=2048, out_features=1024, bias=False)
              (relative_attention_bias): Embedding(32, 16)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=1024, out_features=8192, bias=False)
              (wi_1): Linear(in_features=1024, out_features=8192, bias=False)
     

In [5]:
arg_dev='/kaggle/input/kpm-data/arguments_dev.csv'
arg_train='/kaggle/input/kpm-data/arguments_train.csv'
arg_test='/kaggle/input/kpm-data/arguments_test.csv'
kp_dev='/kaggle/input/kpm-data/key_points_dev.csv'
kp_train='/kaggle/input/kpm-data/key_points_train.csv'
kp_test='/kaggle/input/kpm-data/key_points_test.csv'

In [6]:
def read_csvs(arg_dev, arg_train, arg_test, kp_dev, kp_train, kp_test):
    # Read argument CSVs
    df_arg_dev = pd.read_csv(arg_dev)
    df_arg_train = pd.read_csv(arg_train)
    df_arg_test = pd.read_csv(arg_test)
    
    # Read key points CSVs
    df_kp_dev = pd.read_csv(kp_dev)
    df_kp_train = pd.read_csv(kp_train)
    df_kp_test = pd.read_csv(kp_test)
    
    return df_arg_dev, df_arg_train, df_arg_test, df_kp_dev, df_kp_train, df_kp_test

# Call the function to read CSVs into DataFrames
df_arg_dev, df_arg_train, df_arg_test, df_kp_dev, df_kp_train, df_kp_test = read_csvs(arg_dev, arg_train, arg_test, kp_dev, kp_train, kp_test)

In [7]:
first_row = df_arg_dev.iloc[0]
first_row

arg_id                                                arg_4_0
argument    having a school uniform can reduce bullying as...
topic             We should abandon the use of school uniform
stance                                                     -1
Name: 0, dtype: object

In [8]:
dash_line = '-'.join('' for x in range(100))
# get the passage
argument = first_row['argument']
# get the human question
topic = first_row['topic']
# Configurations
generation_config = GenerationConfig(max_new_tokens=100)
# Encode input:
inputs_encoded = tokenizer(f" <2el>{topic}",return_tensors='pt')
# Model Output:
model_output = model.generate(inputs_encoded["input_ids"].to(device), generation_config=generation_config)[0]
# Decode the output
zero_output = tokenizer.decode(model_output, skip_special_tokens=True)
print("Input: ", argument)
print(dash_line)
print( "English text: ", topic)
print(dash_line)
print("Model Output: ", zero_output)

2024-05-02 06:48:16.683400: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-02 06:48:16.683507: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-02 06:48:16.886454: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Input:  having a school uniform can reduce bullying as students who have no style or cannot afford the latest trends do not stand out.
---------------------------------------------------------------------------------------------------
English text:  We should abandon the use of school uniform
---------------------------------------------------------------------------------------------------
Model Output:  Πρέπει να εγκαταλείψουμε τη χρήση της σχολικής στολής


#Zeroshot translation on Args

In [9]:
# Translation code
def translate_arg_dataframe(df):
    translated_rows = []
    for index, row in df.iterrows():
        argument = row['argument']
        topic = row['topic']
        
        # Translate argument
        inputs_encoded_arg = tokenizer(f" <2el>{argument}", return_tensors='pt')
        model_output_arg = model.generate(inputs_encoded_arg["input_ids"].to(device), generation_config=generation_config)[0]
        translated_argument = tokenizer.decode(model_output_arg, skip_special_tokens=True)
        
        # Translate topic
        inputs_encoded_topic = tokenizer(f" <2el>{topic}", return_tensors='pt')
        model_output_topic = model.generate(inputs_encoded_topic["input_ids"].to(device), generation_config=generation_config)[0]
        translated_topic = tokenizer.decode(model_output_topic, skip_special_tokens=True)
        
        translated_rows.append({'argument': translated_argument, 'topic': translated_topic})
    
    return pd.DataFrame(translated_rows)

# Translate each dataframe
zero_shot_translated_args_dev_df = translate_arg_dataframe(df_arg_dev)
zero_shot_translated_args_train_df = translate_arg_dataframe(df_arg_train)
zero_shot_translated_args_test_df = translate_arg_dataframe(df_arg_test)

# Save translated dataframes to CSV
zero_shot_translated_args_dev_df.to_csv('zero_shot_translated_args_dev.csv', index=False)
zero_shot_translated_args_train_df.to_csv('zero_shot_translated_args_train.csv', index=False)
zero_shot_translated_args_test_df.to_csv('zero_shot_translated_args_test.csv', index=False)

#Zeroshot translation on KPs

In [10]:
def translate_kp_dataframe(df):
    translated_rows = []
    for index, row in df.iterrows():
        keypoint = row['key_point']
        topic = row['topic']
        
        # Translate argument
        inputs_encoded_kp = tokenizer(f" <2el>{keypoint}", return_tensors='pt')
        model_output_kp = model.generate(inputs_encoded_kp["input_ids"].to(device), generation_config=generation_config)[0]
        translated_kp = tokenizer.decode(model_output_kp, skip_special_tokens=True)
        
        # Translate topic
        inputs_encoded_topic = tokenizer(f" <2el>{topic}", return_tensors='pt')
        model_output_topic = model.generate(inputs_encoded_topic["input_ids"].to(device), generation_config=generation_config)[0]
        translated_topic = tokenizer.decode(model_output_topic, skip_special_tokens=True)
        
        translated_rows.append({'key_point': translated_kp, 'topic': translated_topic})
    
    return pd.DataFrame(translated_rows)

# Translate each dataframe
zero_shot_translated_kp_dev_df = translate_kp_dataframe(df_kp_dev)
zero_shot_translated_kp_train_df = translate_kp_dataframe(df_kp_train)
zero_shot_translated_kp_test_df = translate_kp_dataframe(df_kp_test)

# Save translated dataframes to CSV
zero_shot_translated_kp_dev_df.to_csv('zero_shot_translated_kp_dev.csv', index=False)
zero_shot_translated_kp_train_df.to_csv('zero_shot_translated_kp_train.csv', index=False)
zero_shot_translated_kp_test_df.to_csv('zero_shot_translated_kp_test.csv', index=False)